In [1]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..',params['model_dir'])

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [12]:
#preprocessing.download_data(params)
#preprocessing.extract_data(params)
#preprocessing.generate_spectogram_images(params)
preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

## Segund Paso: instanciar el modelo

In [3]:
inputs = tf.keras.layers.Input(shape=(110,480, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 240, 32)       4736      
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 240, 32)       128       
_________________________________________________________________
activation (Activation)      (None, 55, 240, 32)       0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 19, 80, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 48640)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               6226048   
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
__________

## Tercer paso: crear el generador

In [4]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 4632 images.
Data in validation
Found 46 images.


## Cuarto paso: correr el modelo

In [5]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

steps_per_epoch = train_generator.n // params['batch_size']
validation_steps = val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback,tb_callback])

Epoch 1/10
289/289 [==============================] - 27s 93ms/step - loss: 0.4222 - sparse_categorical_accuracy: 0.8659 - val_loss: 0.0308 - val_sparse_categorical_accuracy: 1.0000s - loss: 0.4735 - sparse_categorical_accur - ETA: 3s - loss: 0.4624 
Epoch 2/10
289/289 [==============================] - 27s 92ms/step - loss: 0.0836 - sparse_categorical_accuracy: 0.9743 - val_loss: 0.0122 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/10
289/289 [==============================] - 25s 86ms/step - loss: 0.0407 - sparse_categorical_accuracy: 0.9868 - val_loss: 0.0225 - val_sparse_categorical_accuracy: 1.0000: 2s - loss: 0.0420 - spar
Epoch 4/10
289/289 [==============================] - 28s 97ms/step - loss: 0.0449 - sparse_categorical_accuracy: 0.9855 - val_loss: 0.0067 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/10
288/289 [============================>.] - ETA: 0s - loss: 0.0204 - sparse_categorical_accuracy: 0.9933
Epoch 00005: saving model to ..\checkpoints\tf_ckpt

Consider 

In [6]:
params['batch_size'] = val_generator.n
params['shuffle'] = False
val_generator = data.make_datagenerator(params,mode='validation')

Found 46 images.


In [7]:
Y_pred = net.predict_generator(val_generator, steps=1)
y_pred = np.argmax(Y_pred, axis=1)

In [8]:
print('Confusion Matrix')
print(confusion_matrix(val_generator.data, y_pred))
target_names = ["arranca", "acelera", "pita", "izquierda","frena", "detente", "retrocede", "gira", "avanza", "derecha"]
print(classification_report(val_generator.data, y_pred, target_names=target_names))

Confusion Matrix
[[4 0 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 5 0 0 0]
 [0 0 0 0 0 0 0 4 0 0]
 [0 0 0 0 0 0 0 0 5 0]
 [0 0 0 0 0 0 0 0 0 4]]
              precision    recall  f1-score   support

     arranca       1.00      1.00      1.00         4
     acelera       1.00      1.00      1.00         4
        pita       1.00      1.00      1.00         5
   izquierda       1.00      1.00      1.00         5
       frena       1.00      1.00      1.00         5
     detente       1.00      1.00      1.00         5
   retrocede       1.00      1.00      1.00         5
        gira       1.00      1.00      1.00         4
      avanza       1.00      1.00      1.00         5
     derecha       1.00      1.00      1.00         4

   micro avg       1.00      1.00      1.00        46
   macro avg       1.00      1.00      1.00        46
weighted avg       1.00      1.00      1.00     